In [67]:
import serial
import sys
import time

def establecer_comunicacion_serial(puerto, baudios):
    try:
        ser = serial.Serial(puerto, baudios)
        return ser
    except serial.SerialException as e:
        print(f"No se pudo establecer la comunicación serial: {e}")
        return None

puerto = 'COM3'
baudios = 9600
ser = establecer_comunicacion_serial(puerto, baudios)
time.sleep(3)

if ser is not None:
    print(f"Comunicación serial abierta en {puerto} a {baudios} baudios.")
    try:
        dato = ser.readline()
        if dato:
            print("Dato recibido:", dato.decode('latin1'), end='')
    except KeyboardInterrupt:
        ser.close()
        print("\nComunicación serial cerrada.")
else:
    print("La comunicación serial no se pudo establecer. Verifica el puerto y vuelve a intentarlo.")
    sys.exit(1)
ser.close()

Comunicación serial abierta en COM3 a 9600 baudios.
Dato recibido: (-560,26,-53,255,845,40052,5,25,-120,30096)


In [148]:
import serial
import time
import numpy as np

In [152]:
class SerialCommunication:
    def __init__(self, port, baudrate):
        self.ser = serial.Serial(port, baudrate)
        time.sleep(2)

    def readdata(self):
        if self.ser.isOpen():
            data = self.ser.readline().decode('latin1').strip()
            return data
        else:
            print("Error: La comunicación serial no está abierta.")

    def __close__(self):
        if self.ser.isOpen():
            self.ser.close()
            print("Comunicación serial cerrada.")
        else:
            print("Error: La comunicación serial ya está cerrada.")

sensor = SerialCommunication('COM3', 9600)
datos = sensor.readdata()
print("Datos recibidos:", datos)
sensor.__close__()

Datos recibidos: ,µ(-560,26,-53,255,845,40052,5,25,-120,30096)
Comunicación serial cerrada.


In [131]:
import serial.tools.list_ports
availablePorts = serial.tools.list_ports.comports()
print(availablePorts)

In [150]:
database=np.empty((0,10))

In [153]:
index_i = datos.find("(")
index_f = datos.rfind(")")
fila = datos[index_i : index_f+1]
accel_x1,accel_y1,accel_z1,temp1,pres1,alti1,gyro_x1,gyro_y1,gyro_z1,alti2=eval(fila)

In [154]:
accel_x1/=100
accel_y1/=100
accel_z1/=100
temp1/=10
pres1=101325-pres1
alti1/=100
gyro_x1=(gyro_x1*1000)/8.75
gyro_y1=(gyro_y1*1000)/8.75
gyro_z1=(gyro_z1*1000)/8.75
alti2/=100
rfila= [accel_x1,accel_y1,accel_z1,temp1,pres1,alti1,gyro_x1,gyro_y1,gyro_z1,alti2]


In [155]:
if len(database) == 0:
    database = rfila
else:
    database = np.concatenate((database, rfila), axis=0)



In [156]:
database=np.reshape(database,(-1,10))
print(database)

[[-5.60000000e+00  2.60000000e-01 -5.30000000e-01  2.55000000e+01
   1.00480000e+05  4.00520000e+02  5.71428571e+02  2.85714286e+03
  -1.37142857e+04  3.00960000e+02]]


In [1]:
class Operations:
    def __init__(self,variable,tipo):
        self.variable=variable
        self.tipo=tipo
    def operar(self):
        operaciones = {
            '1': lambda x: x / 100,  # aceleraciones
            '2': lambda x: x / 10,   # temperatura
            '3': lambda x: 101325 - x,  # presion
            '4': lambda x: x / 100,  # altura
            '5': lambda x: (x * 1000) / 8.75,  # gyro
        }

        if self.tipo in operaciones:
            self.variable = operaciones[self.tipo](self.variable)
        else:
            print("Tipo no reconocido")

In [22]:
import pandas as pd

class Database:
    def __init__(self, db_name):
        self.db_name = db_name
        self.tables = {}

    def create_table(self, table_name, headers):
        self.tables[table_name] = pd.DataFrame(columns=headers)

    def insert_data(self, table_name, data_dict):
        if table_name not in self.tables:
            raise ValueError(f"Table '{table_name}' does not exist in the database. Create the table first.")
        new_row = pd.DataFrame(data_dict, index=[0])
        self.tables[table_name] = pd.concat([self.tables[table_name], new_row], ignore_index=True)
        print(f"Data inserted into '{table_name}' table.")

    def get_last_row(self, table_name):
        if table_name not in self.tables:
            raise ValueError(f"Table '{table_name}' does not exist in the database.")
        last_row = self.tables[table_name].iloc[-1]
        return last_row

    def show_tables(self):
        print("Tables in the database:")
        for table_name, df in self.tables.items():
            print(f"Table: {table_name}")
            print(df)

In [23]:
#(-560,26,-53,255,845,40052,5,25,-120,30096)
accel_x=-5.6
accel_y=0.26
accel_z=-0.53
temp1=25.5
pres1=2.55
alti1=400.52
gyro_x=0.02392
gyro_y=0.01223
gyro_z=0.04517
alti2=300.96

In [31]:
my_db = Database("MyDatabase")
headers = ["Ax", "Ay", "Az", "Temperatura", "Presion", "Altura1", "Gx", "Gy", "Gz", "Altura2"]
my_db.create_table("Database", headers)

my_db.insert_data("Database", {"Ax": accel_x , "Ay": accel_y, "Az": accel_z, "Temperatura": temp1, "Presion": pres1, "Altura1": alti1,
                               "Gx": gyro_x, "Gy": gyro_y, "Gz": gyro_z, "Altura2": alti2})
my_db.insert_data("Database", {"Ax": accel_x + 1, "Ay": accel_y + 1, "Az": accel_z + 1, "Temperatura": temp1 + 0.5,
                               "Presion": pres1 + 0.2, "Altura1": alti1 - 10,"Gx": gyro_x - 0.001, "Gy": gyro_y - 0.001,
                               "Gz": gyro_z + 0.001, "Altura2": alti2 - 10})
my_db.insert_data("Database", {"Ax": accel_x + 1, "Ay": accel_y + 1, "Az": accel_z + 1, "Temperatura": temp1 + 0.5,
                               "Presion": pres1 + 0.2, "Altura1": alti1 - 10,"Gx": gyro_x - 0.001, "Gy": gyro_y - 0.001,
                               "Gz": gyro_z + 0.001, "Altura2": alti2 - 10})
my_db.insert_data("Database", {"Ax": accel_x + 1, "Ay": accel_y + 1, "Az": accel_z + 1, "Temperatura": temp1 + 0.5,
                               "Presion": pres1 + 0.2, "Altura1": alti1 - 10,"Gx": gyro_x - 0.001, "Gy": gyro_y - 0.001,
                               "Gz": gyro_z + 0.001, "Altura2": alti2 - 10})
# Obtener la última fila de la tabla "Database"
last_row = my_db.get_last_row("Database")

Data inserted into 'Database' table.
Data inserted into 'Database' table.
Data inserted into 'Database' table.
Data inserted into 'Database' table.


C:\Users\Usuario\AppData\Local\Temp\ipykernel_5156\2767842942.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.tables[table_name] = pd.concat([self.tables[table_name], new_row], ignore_index=True)


In [33]:
class Integrate:
    def __init__(self, a1, a2, t1, t2, s1, v1):
        self.a1 = a1  # Aceleración inicial
        self.a2 = a2  # Aceleración final
        self.t1 = t1  # Tiempo inicial
        self.t2 = t2  # Tiempo final
        self.s1 = s1  # Posición inicial
        self.v1 = v1  # Velocidad inicial
        self.v2 = self.calcular_vf()

    def calcular_vf(self):
        return self.v1 + (self.a2 - self.a1)*0.5*(self.t2-self.t1)

    def calcular_pf(self):
        delta_v = self.v2 - self.v1
        delta_t = self.t2 - self.t1

        posicion = self.s1 + (self.v1 * delta_t) + (0.5 * self.a1 * delta_t**2)
        return posicion

In [34]:
# Ejemplo de uso
try:
    a1 = 2  # Aceleración inicial
    a2 = 3  # Aceleración final
    t1 = 0  # Tiempo inicial
    t2 = 5  # Tiempo final
    s1 = 0  # Posición inicial
    v1 = 0  # Velocidad inicial
    movimiento = Integrate(a1, a2, t1, t2, s1, v1)
    velocidad_instantanea = movimiento.calcular_vf()
    posicion_final = movimiento.calcular_pf()

    print(f"La velocidad instantánea es: {velocidad_instantanea} m/s²")
    print(f"La posición final es: {posicion_final} m")
except ZeroDivisionError:
    print("El tiempo no puede ser cero para calcular la velocidad instantánea.")

La velocidad instantánea es: 2.5 m/s²
La posición final es: 25.0 m
